In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import cv2
import os
from google.colab.patches import cv2_imshow
import xml.etree.ElementTree as ET

In [3]:
!pip install mtcnn
from mtcnn import MTCNN
detector=MTCNN()

     |████████████████████████████████| 2.3MB 5.8MB/s 


In [5]:
# Unzip kaggle data and some other data of incorrectly masked faces obtained from kaggle
!unzip "/content/drive/MyDrive/KaggleData.zip"
!unzip "/content/drive/MyDrive/AdditionalIncorrectFaces.zip"
label_folder="/content/annotations/"
image_folder="/content/images/"
countwithmask=0
countincorrectmask=0
countwithoutmask=0
labelFiles=os.listdir(label_folder)
imageFiles=os.listdir(image_folder)
# The dataset has three labels but we only need two
maskValue={
    'without_mask':1,
    'mask_weared_incorrect':1,
    'with_mask':0
}
correctCount=0
incorrectCount=0
CorrectFeatures=[]
IncorrectFeatures=[]
CorrectLabels=[]
IncorrectLabels=[]
LabelNames=[]
for i in range(len(labelFiles)):
  img=cv2.imread(image_folder+imageFiles[i]) 
  mylabelfile=imageFiles[i].replace(".png","")
  mylabelfile=mylabelfile+".xml"
  tree = ET.parse(label_folder+mylabelfile)
  root = tree.getroot()
  for object in root.findall('object'):
    name=object.find('name').text
    label=maskValue[name]
    # Get face image coordinateds using provided xml file along with the kaggle data
    for bndbox in object.findall("bndbox"):
      xmin=int(bndbox.find('xmin').text)
      xmax=int(bndbox.find('xmax').text)
      ymin=int(bndbox.find('ymin').text)
      ymax=int(bndbox.find('ymax').text)
    if xmax-xmin>=16 and ymax-ymin>=16:
      try:
        myimg=img[ymin:ymax,xmin:xmax,:]
        img_rgb=cv2.cvtColor(myimg,cv2.COLOR_BGR2RGB)
        

        myimg=cv2.resize(myimg,(16,16))
        myimg=np.dot(myimg[...,:3], [0.299, 0.587, 0.144])  # Reshape kaggle data

        if name=="without_mask":
          countwithoutmask=countwithoutmask+1
          IncorrectFeatures.append(myimg)
          IncorrectLabels.append(label)
        elif name=="mask_weared_incorrect":
          countincorrectmask=countincorrectmask+1
          IncorrectFeatures.append(myimg)
          IncorrectLabels.append(label)
        elif name=="with_mask":
          countwithmask=countwithmask+1
          CorrectFeatures.append(myimg)
          CorrectLabels.append(label)
        LabelNames.append(name)
      except:
        print("Error")

# Because kaggle data have very few images of incorrecly masked faces. Lets merge it with some more incorrectly masked images
folder_path="/content/AdditionalIncorrectFaces/"
files=os.listdir(folder_path)
for myfile in files:
  img=cv2.imread(folder_path+myfile)
  reshaped_image=np.dot(img[...,:3], [0.299, 0.587, 0.144])
  cropped_img=cv2.resize(reshaped_image,(16,16))
  cv2_imshow(cropped_img)
  IncorrectFeatures.append(cropped_img)
  IncorrectLabels.append(1)
    

CorrectFeatures=np.asarray(CorrectFeatures)
IncorrectFeatures=np.asarray(IncorrectFeatures)
CorrectLabels=np.asarray(CorrectLabels)
IncorrectLabels=np.asarray(IncorrectLabels)

Archive:  /content/drive/MyDrive/KaggleData.zip
replace annotations/maksssksksss0.xml? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
Archive:  /content/drive/MyDrive/AdditionalIncorrectFaces.zip
replace AdditionalIncorrectFaces/0_0_0 copy 3.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [7]:
print("Correct Data Shape",CorrectFeatures.shape,"Incorrect Data Shape",IncorrectFeatures.shape)

Correct Data Shape (2287, 16, 16) Incorrect Data Shape (550, 16, 16)


In [8]:
cv2_imshow(CorrectFeatures[0])

In [9]:
cv2_imshow(IncorrectFeatures[0])

In [10]:
# Reshape correcly and incorrectly masked faces
CorrectFeatures=np.reshape(CorrectFeatures,(CorrectFeatures.shape[0],CorrectFeatures.shape[1],CorrectFeatures.shape[2],1))
IncorrectFeatures=np.reshape(IncorrectFeatures,(IncorrectFeatures.shape[0],IncorrectFeatures.shape[1],IncorrectFeatures.shape[2],1))

In [11]:
print("Correct Data Shape",CorrectFeatures.shape,"Incorrect Data Shape",IncorrectFeatures.shape)

Correct Data Shape (2287, 16, 16, 1) Incorrect Data Shape (550, 16, 16, 1)


In [13]:
# Save correctly and incorrectly masked features
from numpy import savez_compressed
savez_compressed('/content/drive/MyDrive/MSProjectData/Dataset2/Correct/correct_features.npz', CorrectFeatures)
savez_compressed('/content/drive/MyDrive/MSProjectData/Dataset2/Incorrect/incorrect_features.npz', IncorrectFeatures)